# Trading Signals - SMA

In [31]:
# Import libraries and dependencies
import numpy as np
import pandas as pd
from pathlib import Path
import hvplot
import hvplot.pandas
from IPython.display import Markdown

pd.set_option("display.max_rows", 2000)
pd.set_option("display.max_columns", 2000)
pd.set_option("display.width", 1000)

### Read CSV into Pandas DataFrame

In [32]:
# Specify stock ticker
ticker = 'TSLA'

In [33]:
# Set the file path
filepath = Path("Resources/stock_data.csv")

# Read the CSV located at the file path into a Pandas DataFrame
stock_df = pd.read_csv(filepath, parse_dates=True, infer_datetime_format=True)

stock_df = stock_df.sort_index()
stock_df.drop('Unnamed: 0', inplace=True, axis=1)

stock_df.tail()

,date,open,high,low,close,volume
48,2021-03-15,694.09,713.18,684.0400,707.94,28269001
49,2021-03-16,703.35,707.92,671.0000,676.63,31258483
50,2021-03-17,656.87,703.73,651.0100,701.54,37888931
51,2021-03-18,684.29,689.23,652.0000,653.30,31802623
52,2021-03-19,646.60,657.23,624.6201,654.97,37974884


### Generate a Dual Moving Average Crossover Trading Signal

In [34]:
# Grab just the `date` and `close` from the IEX dataset
signals_df = stock_df.loc[:, ['date', 'close']].copy()

# Set the `date` column as the index
signals_df = signals_df.set_index("date", drop=True)

# Set the short window and long windows
short_window = 50
long_window = 100

# Generate the short and long moving averages (50 and 100 days, respectively)
signals_df['SMA50'] = signals_df['close'].rolling(window=short_window).mean()
signals_df['SMA100'] = signals_df['close'].rolling(window=long_window).mean()
signals_df['Signal'] = 0.0

# Generate the trading signal 0 or 1,
# where 0 is when the SMA50 is under the SMA100, and
# where 1 is when the SMA50 is higher (or crosses over) the SMA100
signals_df['Signal'][short_window:] = np.where(
    signals_df['SMA50'][short_window:] > signals_df['SMA100'][short_window:], 1.0, 0.0
)

# Calculate the points in time at which a position should be taken, 1 or -1
signals_df['Entry/Exit'] = signals_df['Signal'].diff()

# Print the DataFrame
signals_df.tail(10)

,close,SMA50,SMA100,Signal,Entry/Exit
date,,,,,
2021-03-08,562.97,NaN,NaN,0.0,0.0
2021-03-09,673.58,NaN,NaN,0.0,0.0
2021-03-10,668.27,NaN,NaN,0.0,0.0
2021-03-11,699.60,NaN,NaN,0.0,0.0
2021-03-12,693.84,NaN,NaN,0.0,0.0
2021-03-15,707.94,NaN,NaN,0.0,0.0
2021-03-16,676.63,775.858972,NaN,0.0,0.0
2021-03-17,701.54,775.294772,NaN,0.0,0.0
2021-03-18,653.30,773.659672,NaN,0.0,0.0


### Plot Entry and Exit Points of Dual Moving Average Crossover Trading Strategy

In [35]:
# Visualize exit position relative to close price
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['close'].hvplot.scatter(
    color='red',
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize entry position relative to close price
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['close'].hvplot.scatter(
    color='green',
    legend=False,
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize close price for the investment
security_close = signals_df[['close']].hvplot(
    line_color='lightgray',
    ylabel='Price in $',
    width=1000,
    height=400
)

# Visualize moving averages
moving_avgs = signals_df[['SMA50', 'SMA100']].hvplot(
    ylabel='Price in $',
    width=1000,
    height=400
)

# Overlay plots
entry_exit_plot = security_close * moving_avgs * entry * exit
entry_exit_plot.opts(xaxis=None)

:Overlay
   .Curve.I          :Curve   [date]   (close)
   .NdOverlay.I      :NdOverlay   [Variable]
      :Curve   [date]   (value)
   .Scatter.Close.I  :Scatter   [date]   (close)
   .Scatter.Close.II :Scatter   [date]   (close)

### Backtest the Trading Strategy

In [36]:
# Set initial capital
initial_capital = float(100000)

# Set the share size
share_size = 500

# Take a 500 share position where the dual moving average crossover is 1 (SMA50 is greater than SMA100)
signals_df['Position'] = share_size * signals_df['Signal']

# Find the points in time where a 500 share position is bought or sold
signals_df['Entry/Exit Position'] = signals_df['Position'].diff()

# Multiply share price by entry/exit positions and get the cumulatively sum
signals_df['Portfolio Holdings'] = signals_df['close'] * signals_df['Entry/Exit Position'].cumsum()

# Subtract the initial capital by the portfolio holdings to get the amount of liquid cash in the portfolio
signals_df['Portfolio Cash'] = initial_capital - (signals_df['close'] * signals_df['Entry/Exit Position']).cumsum()

# Get the total portfolio value by adding the cash amount by the portfolio holdings (or investments)
signals_df['Portfolio Total'] = signals_df['Portfolio Cash'] + signals_df['Portfolio Holdings']

# Calculate the portfolio daily returns
signals_df['Portfolio Daily Returns'] = signals_df['Portfolio Total'].pct_change()

# Calculate the cumulative returns
signals_df['Portfolio Cumulative Returns'] = (1 + signals_df['Portfolio Daily Returns']).cumprod() - 1

# Print the DataFrame
signals_df.tail(10)

,close,SMA50,SMA100,Signal,Entry/Exit,Position,Entry/Exit Position,Portfolio Holdings,Portfolio Cash,Portfolio Total,Portfolio Daily Returns,Portfolio Cumulative Returns
date,,,,,,,,,,,,
2021-03-08,562.97,NaN,NaN,0.0,0.0,0.0,0.0,0.0,100000.0,100000.0,0.0,0.0
2021-03-09,673.58,NaN,NaN,0.0,0.0,0.0,0.0,0.0,100000.0,100000.0,0.0,0.0
2021-03-10,668.27,NaN,NaN,0.0,0.0,0.0,0.0,0.0,100000.0,100000.0,0.0,0.0
2021-03-11,699.60,NaN,NaN,0.0,0.0,0.0,0.0,0.0,100000.0,100000.0,0.0,0.0
2021-03-12,693.84,NaN,NaN,0.0,0.0,0.0,0.0,0.0,100000.0,100000.0,0.0,0.0
2021-03-15,707.94,NaN,NaN,0.0,0.0,0.0,0.0,0.0,100000.0,100000.0,0.0,0.0
2021-03-16,676.63,775.858972,NaN,0.0,0.0,0.0,0.0,0.0,100000.0,100000.0,0.0,0.0
2021-03-17,701.54,775.294772,NaN,0.0,0.0,0.0,0.0,0.0,100000.0,100000.0,0.0,0.0
2021-03-18,653.30,773.659672,NaN,0.0,0.0,0.0,0.0,0.0,100000.0,100000.0,0.0,0.0


### Plot Entry/Exit Points of Trading Strategy vs. Backtest Results

In [37]:
# Visualize exit position relative to total portfolio value
exit = signals_df[signals_df['Entry/Exit'] == -1.0]['Portfolio Total'].hvplot.scatter(
    color='red',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize entry position relative to total portfolio value
entry = signals_df[signals_df['Entry/Exit'] == 1.0]['Portfolio Total'].hvplot.scatter(
    color='green',
    legend=False,
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Visualize total portoflio value for the investment
total_portfolio_value = signals_df[['Portfolio Total']].hvplot(
    line_color='lightgray',
    ylabel='Total Portfolio Value',
    width=1000,
    height=400
)

# Overlay plots
portfolio_entry_exit_plot = total_portfolio_value * entry * exit
portfolio_entry_exit_plot.opts(xaxis=None)

:Overlay
   .Curve.I                    :Curve   [date]   (Portfolio Total)
   .Scatter.Portfolio_Total.I  :Scatter   [date]   (Portfolio Total)
   .Scatter.Portfolio_Total.II :Scatter   [date]   (Portfolio Total)

### Prepare Portfolio Evaluation Metrics DataFrame

In [38]:
# Prepare DataFrame for metrics
metrics = [
    'Annual Return',
    'Cumulative Returns',
    'Annual Volatility',
    'Sharpe Ratio',
    'Sortino Ratio']

columns = ['Backtest']

# Initialize the DataFrame with index set to evaluation metrics and column as `Backtest` (just like PyFolio)
portfolio_evaluation_df = pd.DataFrame(index=metrics, columns=columns)
portfolio_evaluation_df

,Backtest
Annual Return,NaN
Cumulative Returns,NaN
Annual Volatility,NaN
Sharpe Ratio,NaN
Sortino Ratio,NaN


### Calculate and Assign Portfolio Evaluation Metrics

In [39]:
# Calculate cumulative return
portfolio_evaluation_df.loc['Cumulative Returns'] = signals_df['Portfolio Cumulative Returns'][-1]

# Calculate annualized return
portfolio_evaluation_df.loc['Annual Return'] = (
    signals_df['Portfolio Daily Returns'].mean() * 252
)

# Calculate annual volatility
portfolio_evaluation_df.loc['Annual Volatility'] = (
    signals_df['Portfolio Daily Returns'].std() * np.sqrt(252)
)

# Calculate Sharpe Ratio
portfolio_evaluation_df.loc['Sharpe Ratio'] = (
    signals_df['Portfolio Daily Returns'].mean() * 252) / (
    signals_df['Portfolio Daily Returns'].std() * np.sqrt(252)
)

# Calculate Downside Return
sortino_ratio_df = signals_df[['Portfolio Daily Returns']].copy()
sortino_ratio_df.loc[:,'Downside Returns'] = 0

target = 0
mask = sortino_ratio_df['Portfolio Daily Returns'] < target
sortino_ratio_df.loc[mask, 'Downside Returns'] = sortino_ratio_df['Portfolio Daily Returns']**2
portfolio_evaluation_df

# Calculate Sortino Ratio
down_stdev = np.sqrt(sortino_ratio_df['Downside Returns'].mean()) * np.sqrt(252)
expected_return = sortino_ratio_df['Portfolio Daily Returns'].mean() * 252
sortino_ratio = expected_return/down_stdev

portfolio_evaluation_df.loc['Sortino Ratio'] = sortino_ratio
portfolio_evaluation_df.head()

ipykernel_launcher:17: RuntimeWarning: invalid value encountered in double_scalars
ipykernel_launcher:32: RuntimeWarning: invalid value encountered in double_scalars


,Backtest
Annual Return,0
Cumulative Returns,0
Annual Volatility,0
Sharpe Ratio,NaN
Sortino Ratio,NaN


### Prepare Trade Evaluation Metrics DataFrame

In [40]:
# Initialize trade evaluation DataFrame with columns
trade_evaluation_df = pd.DataFrame(
    columns=[
        'Stock', 
        'Entry Date', 
        'Exit Date', 
        'Shares', 
        'Entry Share Price', 
        'Exit Share Price', 
        'Entry Portfolio Holding', 
        'Exit Portfolio Holding', 
        'Profit/Loss']
)

trade_evaluation_df

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss


### Generating Trade Evaluation Metrics

In [41]:
# Initialize iterative variables
entry_date = ''
exit_date = ''
entry_portfolio_holding = 0
exit_portfolio_holding = 0
share_size = 0
entry_share_price = 0
exit_share_price = 0

# Loop through signal DataFrame
# If `Entry/Exit` is 1, set entry trade metrics
# Else if `Entry/Exit` is -1, set exit trade metrics and calculate profit,
# Then append the record to the trade evaluation DataFrame
for index, row in signals_df.iterrows():
    if row['Entry/Exit'] == 1:
        entry_date = index
        entry_portfolio_holding = abs(row['Portfolio Holdings'])
        share_size = row['Entry/Exit Position']
        entry_share_price = row['close']

    elif row['Entry/Exit'] == -1:
        exit_date = index
        exit_portfolio_holding = abs(row['close'] * row['Entry/Exit Position'])
        exit_share_price = row['close']
        profit_loss =  entry_portfolio_holding - exit_portfolio_holding
        trade_evaluation_df = trade_evaluation_df.append(
            {
                'Stock': (f"{ticker}"),
                'Entry Date': entry_date,
                'Exit Date': exit_date,
                'Shares': share_size,
                'Entry Share Price': entry_share_price,
                'Exit Share Price': exit_share_price,
                'Entry Portfolio Holding': entry_portfolio_holding,
                'Exit Portfolio Holding': exit_portfolio_holding,
                'Profit/Loss': profit_loss
            },
            ignore_index=True)

# Print the DataFrame
trade_evaluation_df

,Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss


---

### Create Hvplot Line Chart of Closing, SMA50, and SMA100

In [42]:
price_df = signals_df[['close', 'SMA50', 'SMA100']]
price_chart = price_df.hvplot.line()
price_chart.opts(xaxis=None)

:NdOverlay   [Variable]
   :Curve   [date]   (value)

### Create Hvplot Table of Portfolio Metrics

In [43]:
portfolio_evaluation_df.reset_index(inplace=True)
portfolio_evaluation_table = portfolio_evaluation_df.hvplot.table()
portfolio_evaluation_table

:Table   [index,Backtest]

### Create Hvplot Table of Trade Metrics

In [44]:
trade_evaluation_table = trade_evaluation_df.hvplot.table()
trade_evaluation_table

:Table   [Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss]

### Build a Dashboard with `hvplot`

In [45]:
# Assemble dashboard visualization
display(Markdown("# Trading Dashboard"))
display(price_chart)
portfolio_evaluation_table + trade_evaluation_table

# Trading Dashboard

:NdOverlay   [Variable]
   :Curve   [date]   (value)

:Layout
   .Table.I  :Table   [index,Backtest]
   .Table.II :Table   [Stock,Entry Date,Exit Date,Shares,Entry Share Price,Exit Share Price,Entry Portfolio Holding,Exit Portfolio Holding,Profit/Loss]